In [9]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.classical import expr

def get_dynamic_CNOT_Circuit(num_qubit):

    num_ancilla = num_qubit - 2
    num_ancilla_pair = int(num_ancilla / 2)

    qr = QuantumRegister(num_qubit)
    cr1 = ClassicalRegister(num_ancilla_pair, name="cr1")
    cr2 = ClassicalRegister(num_ancilla - num_ancilla_pair, name="cr2")
    cr3 = ClassicalRegister(2, name="cr3")
    qc = QuantumCircuit(qr, cr1, cr2, cr3)

    qc.h(0)
    qc.barrier()

    qc.cx(0,1)

    for i in range(num_ancilla_pair):
        qc.h(2+2*i)
        qc.cx(2+2*i, 2+2*i+1)

    for i in range(num_ancilla_pair + 1):
        qc.cx(1+2*i, 1+2*i+1)
    for i in range(1, num_ancilla_pair + 2):
        qc.h(2*i-1)

    for i in range(1, num_ancilla_pair + 2):
        qc.measure(2 * i - 1, cr2[i - 1])
        if  i == 1:
            parity_control = expr.lift(cr2[i - 1])
        else:
            parity_control = expr.bit_xor(cr2[i - 1], parity_control)

    for i in range(num_ancilla_pair):
        qc.measure(2 * i + 2, cr1[i])
        if  i == 0:
            parity_target = expr.lift(cr1[i])
        else:
            parity_target = expr.bit_xor(cr1[i], parity_target)

        with qc.if_test(parity_control):
            qc.z(0)

        with qc.if_test(parity_target):
            qc.x(-1)

        return qc

In [10]:
qc = get_dynamic_CNOT_Circuit(num_qubit=7)
qc.draw(output="text")

┌───┐ ░                                                               »
 q3_0: ┤ H ├─░───■───────────────────────────────────────────────────────────»
       └───┘ ░ ┌─┴─┐          ┌───┐   ┌─┐                                    »
 q3_1: ──────░─┤ X ├───────■──┤ H ├───┤M├────────────────────────────────────»
             ░ ├───┤     ┌─┴─┐└───┘┌─┐└╥┘                                    »
 q3_2: ──────░─┤ H ├──■──┤ X ├─────┤M├─╫─────────────────────────────────────»
             ░ └───┘┌─┴─┐└───┘┌───┐└╥┘ ║ ┌─┐                                 »
 q3_3: ──────░──────┤ X ├──■──┤ H ├─╫──╫─┤M├─────────────────────────────────»
             ░ ┌───┐└───┘┌─┴─┐└───┘ ║  ║ └╥┘                                 »
 q3_4: ──────░─┤ H ├──■──┤ X ├──────╫──╫──╫──────────────────────────────────»
             ░ └───┘┌─┴─┐└───┘┌───┐ ║  ║  ║ ┌─┐                              »
 q3_5: ──────░──────┤ X ├──■──┤ H ├─╫──╫──╫─┤M├──────────────────────────────»
             ░      └───┘┌─┴─┐└───┘ ║  ║  ║ └╥┘┌───────────── ┌───┐ ───────┐ »
 q3_6: ──────░───────────┤ X ├──────╫──╫──╫──╫─┤ If-0 cr1[0]  ┤ X ├  End-0 ├─»
             ░           └───┘      ║  ║  ║  ║ └──────╥────── └───┘ ───────┘ »
                                    ║  ║  ║  ║    ┌───╨────┐                 »
cr1: 2/═════════════════════════════╩══╬══╬══╬════╡ [expr] ╞═════════════════»
                                    0  ║  ║  ║    └────────┘                 »
cr2: 3/════════════════════════════════╩══╩══╩═══════════════════════════════»
                                       0  1  2                               »
cr3: 2/══════════════════════════════════════════════════════════════════════»
                                                                             »
«       ┌───────────────────────────────── ┌───┐ ───────┐ 
« q3_0: ┤ If-0 cr2[2] ^ (cr2[1] ^ cr2[0])  ┤ Z ├  End-0 ├─
«       └────────────────╥──────────────── └───┘ ───────┘ 
« q3_1: ─────────────────╫────────────────────────────────
«                        ║                                
« q3_2: ─────────────────╫────────────────────────────────
«                        ║                                
« q3_3: ─────────────────╫────────────────────────────────
«                        ║                                
« q3_4: ─────────────────╫────────────────────────────────
«                        ║                                
« q3_5: ─────────────────╫────────────────────────────────
«                        ║                                
« q3_6: ─────────────────╫────────────────────────────────
«                        ║                                
«cr1: 2/═════════════════╬════════════════════════════════
«                    ┌───╨────┐                           
«cr2: 3/═════════════╡ [expr] ╞═══════════════════════════
«                    └────────┘                           
«cr3: 2/══════════════════════════════════════════════════
«

In [11]:
max_num_qubit = 41

qc_list = []
num_qubit_list = list(range(7, max_num_qubit + 1, 2))
for num_qubit in num_qubit_list:
    qc_list.append(get_dynamic_CNOT_Circuit(num_qubit))

In [12]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService

backend_name = "ibm_brisbane"
service = QiskitRuntimeService()
backend = service.get_backend(backend_name)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

qc_transpiled_list = pm.run(qc_list)

/tmp/ipykernel_221272/4212820681.py:6: DeprecationWarning: The get_backend() method is deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use backend() instead.
  backend = service.get_backend(backend_name)


In [13]:
from qiskit_ibm_runtime import SamplerV2 as Sampler 

sampler = Sampler(backend=backend)
job = sampler.run(qc_transpiled_list)
print(job.job_id())

/tmp/ipykernel_221272/4151046348.py:3: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  sampler = Sampler(backend=backend)


csxftehfhyd0008k9vvg


In [ ]:
import matplotlib.pyplot as plt
from qiskit_ibm_runtime import QiskitRuntimeService

job_id = "csxftehfhyd0008k9vvg"
result = job.result()

list_bell = []
list_other = []
for i in range(0, len(qc_list)):
    data.result[i + 1].data
    counts = data.cr3.get_counts()
    total_counts = data.cr3.num_shots

    prob_Bell = (counts['00'] + counts['11']) / total_counts

    list_bell.append(prob_Bell)
    list_other.append(1 - prob_Bell)

plt.plot(num_qubit_list,list_bell, '--o', label ="00 or 11")
plt.plot(num_qubit_list,list_other, '--.^', label = "other")
plt.xlabel("Number of Qubits")
plt.ylabel("Probability")
plt.legend()